In [1]:
# Disable tensorflow warnings (program uses Pytorch)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
# Disable hugging face warnings
import warnings
warnings.filterwarnings("ignore")
# Import the required libraries
import requests
from PIL import Image
from io import BytesIO
from transformers import AutoProcessor, BlipForConditionalGeneration
from bs4 import BeautifulSoup

In [2]:
# Initialize the processor and model
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [3]:
# Provide a URL page to scrape
url = "https://en.wikipedia.org/wiki/Google"

In [4]:
# Download the page
response = requests.get(url)

In [5]:
# Parse the page with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
# Find all the img elements
img_elements = soup.find_all("img")

In [7]:
# Open a file to write the captions
with open("captions.txt", "w") as caption_file:
    # Iterate over each img element
    for img_element in img_elements:
        img_url = img_element.get("src")
        # Skip if the image is an SVG or too small (e.g., icon)
        if "svg" in img_url or "1x1" in img_url:
            continue
        # Correct the URL if it's malformed
        if img_url.startswith("//"):
            img_url = "https:" + img_url
        elif not img_url.startswith("http://") and not img_url.startswith("https://"):
            # Skip URLs that don't start with http:// or https://
            continue
        try:
            # Download the image
            response = requests.get(img_url)
            # Convert the image data to a PIL image
            raw_image = Image.open(BytesIO(response.content))
            if raw_image.size[0] * raw_image.size[1] < 400:
                # Skip very small images
                continue
            raw_image = raw_image.convert("RGB")
            # Process the image
            inputs = processor(raw_image, return_tensors="pt")
            # Generate a caption for the image
            out = model.generate(**inputs, max_new_tokens=50)
            # Decode the generated tokens to text
            caption = processor.decode(out[0], skip_special_tokens=True)
            # Write the caption to the file, prepended by the image URL
            caption_file.write(f"{img_url}: {caption}\n")
        except Exception as e:
            print(f"Error processing image {img_url}: {e}")
            continue